# Implementing Dense Convolutional Network (DenseNet) on Cifar-10
 



In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from keras import backend as K
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [0]:
#Google drive authentication

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
compression = 0.8
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Resize CIFAR10 Data to 24*24 images
if K.image_dim_ordering() == 'th':
    x_train_24 = np.array([cv2.resize(img.transpose(1,2,0), (24,24)).transpose(2,0,1) for img in x_train[:50000,:,:,:]])
    x_test_24 = np.array([cv2.resize(img.transpose(1,2,0), (24,24)).transpose(2,0,1) for img in x_test[:10000,:,:,:]])
else:
    x_train_24 = np.array([cv2.resize(img, (24,24)) for img in x_train[:50000,:,:,:]])
    x_test_24 = np.array([cv2.resize(img, (24,24)) for img in x_test[:10000,:,:,:]])

img_height_24, img_width_24, channel = x_train_24.shape[1],x_train_24.shape[2],x_train_24.shape[3]
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 10s 0us/step


In [0]:
#Data augmentation on 24*24 images
image_gen_24 = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

image_gen_24.fit(x_train_24, augment=True)


#Data augmentation on 32*32 images
image_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

image_gen.fit(x_train, augment=True)



In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    pooling = GlobalAveragePooling2D()(relu)
    output = Dense(num_classes, activation='softmax')(pooling)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 16

# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

input = Input(shape=(None,None,channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, 32, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, 48, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition, 64, dropout_rate)
output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 2 648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 2 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 2 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (


activation_49 (Activation)      (None, None, None, 5 0           batch_normalization_49[0][0]     
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, None, None, 3 188442      activation_49[0][0]              
__________________________________________________________________________________________________
dropout_49 (Dropout)            (None, None, None, 3 0           conv2d_50[0][0]                  
__________________________________________________________________________________________________
concatenate_47 (Concatenate)    (None, None, None, 5 0           concatenate_46[0][0]             
                                                                 dropout_49[0][0]                 
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, None, None, 5 2356        concatenate_47[0][0]             
_________

In [0]:
## Run the model on 24*24 image size for the first 15 epochs

from google.colab import files

learning_rate = 0.005

#Training the model
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model.fit_generator(image_gen_24.flow(x_train_24, y_train, batch_size=batch_size),epochs=15,verbose=1,validation_data=(x_test_24, y_test),callbacks=callbacks_list)

#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model"+str(i)+".json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights"+str(i)+".h5")
print("Saved model to drive")

Epoch 1/15
391/391 [==============================] - 377s 965ms/step - loss: 1.8747 - acc: 0.3079 - val_loss: 2.4868 - val_acc: 0.2569

Epoch 00001: val_acc improved from -inf to 0.25690, saving model to weights-improvement-01-0.26.hdf5
Epoch 2/15
192/391 [=============>................] - ETA: 2:48 - loss: 1.6214 - acc: 0.3956

391/391 [==============================] - 353s 903ms/step - loss: 1.5630 - acc: 0.4189 - val_loss: 2.3124 - val_acc: 0.4020

Epoch 00002: val_acc improved from 0.25690 to 0.40200, saving model to weights-improvement-02-0.40.hdf5
Epoch 3/15
258/391 [==================>...........] - ETA: 1:52 - loss: 1.3982 - acc: 0.4879

391/391 [==============================] - 353s 903ms/step - loss: 1.3640 - acc: 0.5025 - val_loss: 2.1791 - val_acc: 0.4217

Epoch 00003: val_acc improved from 0.40200 to 0.42170, saving model to weights-improvement-03-0.42.hdf5
Epoch 4/15
275/391 [====================>.........] - ETA: 1:37 - loss: 1.1982 - acc: 0.5672

391/391 [==============================] - 353s 902ms/step - loss: 1.1794 - acc: 0.5741 - val_loss: 1.7609 - val_acc: 0.5100

Epoch 00004: val_acc improved from 0.42170 to 0.51000, saving model to weights-improvement-04-0.51.hdf5
Epoch 5/15
279/391 [====================>.........] - ETA: 1:34 - loss: 1.0605 - acc: 0.6192

391/391 [==============================] - 353s 904ms/step - loss: 1.0562 - acc: 0.6230 - val_loss: 1.3594 - val_acc: 0.6133

Epoch 00005: val_acc improved from 0.51000 to 0.61330, saving model to weights-improvement-05-0.61.hdf5
Epoch 6/15
280/391 [====================>.........] - ETA: 1:33 - loss: 0.9907 - acc: 0.6475

...

391/391 [==============================] - 352s 901ms/step - loss: 0.6562 - acc: 0.7730 - val_loss: 0.8181 - val_acc: 0.7416

Epoch 00012: val_acc did not improve from 0.75260
Epoch 13/15
343/391 [=========================>....] - ETA: 40s - loss: 0.6295 - acc: 0.7797

391/391 [==============================] - 353s 903ms/step - loss: 0.6284 - acc: 0.7800 - val_loss: 0.9336 - val_acc: 0.7164

Epoch 00013: val_acc did not improve from 0.75260
Epoch 14/15
356/391 [==========================>...] - ETA: 29s - loss: 0.6035 - acc: 0.7907

391/391 [==============================] - 351s 898ms/step - loss: 0.6005 - acc: 0.7915 - val_loss: 0.7804 - val_acc: 0.7666

Epoch 00014: val_acc improved from 0.75260 to 0.76660, saving model to weights-improvement-14-0.77.hdf5
Epoch 15/15
299/391 [=====================>........] - ETA: 1:17 - loss: 0.5777 - acc: 0.7991

391/391 [==============================] - 354s 906ms/step - loss: 0.5790 - acc: 0.7990 - val_loss: 1.2133 - val_acc: 0.6754

Epoch 00015: val_acc did not improve from 0.76660


NameError: ignored

In [0]:
#Saving the weights of first 15 epochs in google drive - As the 'save module' gave an error in the previous cell
#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model1.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights1.h5")
print("Saved model to drive")

Saved model to drive


In [0]:
# Training the model from epoch 16 to 25
i = 2
learning_rate = 0.005

# Load model from drive
json_file = open("drive/assignment4/trial-6/model"+str((i-1))+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("drive/assignment4/trial-6/model_weights"+str((i-1))+".h5")
print("Loaded model from disk")

# Resume training
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),epochs=10,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks_list)

#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model"+str(i)+".json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights"+str(i)+".h5")
print("Saved model to drive")

Loaded model from disk
Epoch 1/10
391/391 [==============================] - 576s 1s/step - loss: 0.5110 - acc: 0.8224 - val_loss: 0.7657 - val_acc: 0.7783

Epoch 00001: val_acc improved from -inf to 0.77830, saving model to weights-improvement-01-0.78.hdf5
Epoch 2/10
155/391 [==========>...................] - ETA: 5:07 - loss: 0.4642 - acc: 0.8388

391/391 [==============================] - 549s 1s/step - loss: 0.4622 - acc: 0.8399 - val_loss: 0.5114 - val_acc: 0.8368

Epoch 00002: val_acc improved from 0.77830 to 0.83680, saving model to weights-improvement-02-0.84.hdf5
Epoch 3/10
251/391 [==================>...........] - ETA: 3:02 - loss: 0.4460 - acc: 0.8457

391/391 [==============================] - 548s 1s/step - loss: 0.4504 - acc: 0.8442 - val_loss: 0.6516 - val_acc: 0.8045

Epoch 00003: val_acc did not improve from 0.83680
Epoch 4/10
333/391 [========================>.....] - ETA: 1:15 - loss: 0.4228 - acc: 0.8525

391/391 [==============================] - 549s 1s/step - loss: 0.4244 - acc: 0.8518 - val_loss: 1.0522 - val_acc: 0.6996

Epoch 00004: val_acc did not improve from 0.83680
Epoch 5/10
358/391 [==========================>...] - ETA: 43s - loss: 0.4091 - acc: 0.8590

391/391 [==============================] - 549s 1s/step - loss: 0.4089 - acc: 0.8590 - val_loss: 0.6838 - val_acc: 0.8065

Epoch 00005: val_acc did not improve from 0.83680
Epoch 6/10
366/391 [===========================>..] - ETA: 32s - loss: 0.3960 - acc: 0.8636

391/391 [==============================] - 549s 1s/step - loss: 0.3956 - acc: 0.8640 - val_loss: 0.6732 - val_acc: 0.8149

Epoch 00006: val_acc did not improve from 0.83680
Epoch 7/10
368/391 [===========================>..] - ETA: 29s - loss: 0.3795 - acc: 0.8681

391/391 [==============================] - 548s 1s/step - loss: 0.3796 - acc: 0.8682 - val_loss: 0.5126 - val_acc: 0.8448

Epoch 00007: val_acc improved from 0.83680 to 0.84480, saving model to weights-improvement-07-0.84.hdf5
Epoch 8/10
305/391 [======================>.......] - ETA: 1:52 - loss: 0.3636 - acc: 0.8737

391/391 [==============================] - 548s 1s/step - loss: 0.3655 - acc: 0.8735 - val_loss: 0.5120 - val_acc: 0.8465

Epoch 00008: val_acc improved from 0.84480 to 0.84650, saving model to weights-improvement-08-0.85.hdf5
Epoch 9/10
289/391 [=====================>........] - ETA: 2:13 - loss: 0.3562 - acc: 0.8777

391/391 [==============================] - 549s 1s/step - loss: 0.3564 - acc: 0.8773 - val_loss: 0.5423 - val_acc: 0.8409

Epoch 00009: val_acc did not improve from 0.84650
Epoch 10/10
343/391 [=========================>....] - ETA: 1:02 - loss: 0.3416 - acc: 0.8806

391/391 [==============================] - 549s 1s/step - loss: 0.3414 - acc: 0.8807 - val_loss: 0.4813 - val_acc: 0.8545

Epoch 00010: val_acc improved from 0.84650 to 0.85450, saving model to weights-improvement-10-0.85.hdf5
Saved model to drive


In [0]:
# Training the model from epoch 26 to 35
i = 3
learning_rate = 0.001 #Reduced the learning rate from 0.005 to 0.001

# Load model from drive
json_file = open("drive/assignment4/trial-6/model"+str((i-1))+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("drive/assignment4/trial-6/model_weights"+str((i-1))+".h5")
print("Loaded model from disk")

#Resume training
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),epochs=10,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks_list)

#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model"+str(i)+".json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights"+str(i)+".h5")
print("Saved model to drive")

Loaded model from disk
Epoch 1/10
391/391 [==============================] - 569s 1s/step - loss: 0.2667 - acc: 0.9067 - val_loss: 0.3536 - val_acc: 0.8949

Epoch 00001: val_acc improved from 0.85450 to 0.89490, saving model to weights-improvement-01-0.89.hdf5
Epoch 2/10
154/391 [==========>...................] - ETA: 5:09 - loss: 0.2434 - acc: 0.9150

391/391 [==============================] - 550s 1s/step - loss: 0.2462 - acc: 0.9143 - val_loss: 0.3523 - val_acc: 0.8936

Epoch 00002: val_acc did not improve from 0.89490
Epoch 3/10
303/391 [======================>.......] - ETA: 1:55 - loss: 0.2365 - acc: 0.9182

391/391 [==============================] - 549s 1s/step - loss: 0.2389 - acc: 0.9166 - val_loss: 0.3713 - val_acc: 0.8894

Epoch 00003: val_acc did not improve from 0.89490
Epoch 4/10
349/391 [=========================>....] - ETA: 54s - loss: 0.2314 - acc: 0.9186

391/391 [==============================] - 549s 1s/step - loss: 0.2310 - acc: 0.9189 - val_loss: 0.4145 - val_acc: 0.8841

Epoch 00004: val_acc did not improve from 0.89490
Epoch 5/10
363/391 [==========================>...] - ETA: 36s - loss: 0.2251 - acc: 0.9215

391/391 [==============================] - 550s 1s/step - loss: 0.2247 - acc: 0.9215 - val_loss: 0.3291 - val_acc: 0.9029

Epoch 00005: val_acc improved from 0.89490 to 0.90290, saving model to weights-improvement-05-0.90.hdf5
Epoch 6/10
304/391 [======================>.......] - ETA: 1:53 - loss: 0.2193 - acc: 0.9232

391/391 [==============================] - 550s 1s/step - loss: 0.2192 - acc: 0.9232 - val_loss: 0.2975 - val_acc: 0.9100

Epoch 00006: val_acc improved from 0.90290 to 0.91000, saving model to weights-improvement-06-0.91.hdf5
Epoch 7/10
289/391 [=====================>........] - ETA: 2:13 - loss: 0.2199 - acc: 0.9225

391/391 [==============================] - 550s 1s/step - loss: 0.2200 - acc: 0.9227 - val_loss: 0.3114 - val_acc: 0.9059

Epoch 00007: val_acc did not improve from 0.91000
Epoch 8/10
345/391 [=========================>....] - ETA: 1:00 - loss: 0.2111 - acc: 0.9260

391/391 [==============================] - 551s 1s/step - loss: 0.2112 - acc: 0.9258 - val_loss: 0.3114 - val_acc: 0.9072

Epoch 00008: val_acc did not improve from 0.91000
Epoch 9/10
362/391 [==========================>...] - ETA: 37s - loss: 0.2040 - acc: 0.9273

391/391 [==============================] - 550s 1s/step - loss: 0.2058 - acc: 0.9269 - val_loss: 0.3335 - val_acc: 0.9030

Epoch 00009: val_acc did not improve from 0.91000
Epoch 10/10
366/391 [===========================>..] - ETA: 32s - loss: 0.2043 - acc: 0.9291

391/391 [==============================] - 549s 1s/step - loss: 0.2049 - acc: 0.9291 - val_loss: 0.3556 - val_acc: 0.8986

Epoch 00010: val_acc did not improve from 0.91000
Saved model to drive


In [0]:
# Training the model from epoch 36 to 45
i = 4
learning_rate = 0.001

#Load model from drive
json_file = open("drive/assignment4/trial-6/model"+str((i-1))+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("drive/assignment4/trial-6/model_weights"+str((i-1))+".h5")
print("Loaded model from disk")

# Resume training
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),epochs=10,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks_list)

#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model"+str(i)+".json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights"+str(i)+".h5")
print("Saved model to drive")

Loaded model from disk
Epoch 1/10
391/391 [==============================] - 569s 1s/step - loss: 0.2006 - acc: 0.9302 - val_loss: 0.3703 - val_acc: 0.8937

Epoch 00001: val_acc did not improve from 0.91000
Epoch 2/10
183/391 [=============>................] - ETA: 4:32 - loss: 0.1925 - acc: 0.9323

391/391 [==============================] - 549s 1s/step - loss: 0.1946 - acc: 0.9320 - val_loss: 0.3314 - val_acc: 0.9037

Epoch 00002: val_acc did not improve from 0.91000
Epoch 3/10
312/391 [======================>.......] - ETA: 1:43 - loss: 0.1904 - acc: 0.9326

391/391 [==============================] - 549s 1s/step - loss: 0.1919 - acc: 0.9322 - val_loss: 0.3154 - val_acc: 0.9083

Epoch 00003: val_acc did not improve from 0.91000
Epoch 4/10
352/391 [==========================>...] - ETA: 51s - loss: 0.1893 - acc: 0.9336

391/391 [==============================] - 549s 1s/step - loss: 0.1898 - acc: 0.9336 - val_loss: 0.3076 - val_acc: 0.9088

Epoch 00004: val_acc did not improve from 0.91000
Epoch 5/10
364/391 [==========================>...] - ETA: 35s - loss: 0.1851 - acc: 0.9336

391/391 [==============================] - 549s 1s/step - loss: 0.1849 - acc: 0.9336 - val_loss: 0.3162 - val_acc: 0.9092

Epoch 00005: val_acc did not improve from 0.91000
Epoch 6/10
368/391 [===========================>..] - ETA: 30s - loss: 0.1825 - acc: 0.9348

391/391 [==============================] - 549s 1s/step - loss: 0.1818 - acc: 0.9352 - val_loss: 0.3260 - val_acc: 0.9095

Epoch 00006: val_acc did not improve from 0.91000
Epoch 7/10
369/391 [===========================>..] - ETA: 28s - loss: 0.1842 - acc: 0.9341

391/391 [==============================] - 549s 1s/step - loss: 0.1837 - acc: 0.9345 - val_loss: 0.2960 - val_acc: 0.9121

Epoch 00007: val_acc improved from 0.91000 to 0.91210, saving model to weights-improvement-07-0.91.hdf5
Epoch 8/10
306/391 [======================>.......] - ETA: 1:51 - loss: 0.1810 - acc: 0.9370

391/391 [==============================] - 549s 1s/step - loss: 0.1814 - acc: 0.9365 - val_loss: 0.3160 - val_acc: 0.9113

Epoch 00008: val_acc did not improve from 0.91210
Epoch 9/10
350/391 [=========================>....] - ETA: 53s - loss: 0.1760 - acc: 0.9377

391/391 [==============================] - 549s 1s/step - loss: 0.1777 - acc: 0.9371 - val_loss: 0.3697 - val_acc: 0.9017

Epoch 00009: val_acc did not improve from 0.91210
Epoch 10/10
362/391 [==========================>...] - ETA: 37s - loss: 0.1719 - acc: 0.9399

391/391 [==============================] - 549s 1s/step - loss: 0.1711 - acc: 0.9401 - val_loss: 0.3362 - val_acc: 0.9081

Epoch 00010: val_acc did not improve from 0.91210
Saved model to drive


In [0]:
# Training the model from epoch 46 to 50
i = 5
learning_rate = 0.001

#Load Model from drive
json_file = open("drive/assignment4/trial-6/model"+str((i-1))+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("drive/assignment4/trial-6/model_weights"+str((i-1))+".h5")
print("Loaded model from disk")

# Resume training
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=learning_rate),metrics=['accuracy'])
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),epochs=5,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks_list)

#Save json and weights to drive
model_json = model.to_json()
with open("drive/assignment4/trial-6/model"+str(i)+".json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/assignment4/trial-6/model_weights"+str(i)+".h5")
print("Saved model to drive")

Loaded model from disk
Epoch 1/5
391/391 [==============================] - 573s 1s/step - loss: 0.1748 - acc: 0.9369 - val_loss: 0.3336 - val_acc: 0.9103

Epoch 00001: val_acc did not improve from 0.91210
Epoch 2/5
185/391 [=============>................] - ETA: 4:30 - loss: 0.1631 - acc: 0.9408

391/391 [==============================] - 550s 1s/step - loss: 0.1680 - acc: 0.9401 - val_loss: 0.3674 - val_acc: 0.9024

Epoch 00002: val_acc did not improve from 0.91210
Epoch 3/5
314/391 [=======================>......] - ETA: 1:40 - loss: 0.1651 - acc: 0.9416

391/391 [==============================] - 550s 1s/step - loss: 0.1649 - acc: 0.9413 - val_loss: 0.3816 - val_acc: 0.9016

Epoch 00003: val_acc did not improve from 0.91210
Epoch 4/5
354/391 [==========================>...] - ETA: 48s - loss: 0.1602 - acc: 0.9425

391/391 [==============================] - 550s 1s/step - loss: 0.1617 - acc: 0.9423 - val_loss: 0.3373 - val_acc: 0.9071

Epoch 00004: val_acc did not improve from 0.91210
Epoch 5/5
366/391 [===========================>..] - ETA: 32s - loss: 0.1587 - acc: 0.9443

391/391 [==============================] - 550s 1s/step - loss: 0.1588 - acc: 0.9443 - val_loss: 0.3051 - val_acc: 0.9182

Epoch 00005: val_acc improved from 0.91210 to 0.91820, saving model to weights-improvement-05-0.92.hdf5
Saved model to drive


#Accuracy : 0.9182

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 49s 5ms/step
Test loss: 0.30512408489361403
Test accuracy: 0.9182
